In [26]:
import pandas as pd
import numpy as np
import statsmodels as stats 
import scipy 
import pingouin as pg
from tabulate import tabulate
from texttable import Texttable
import latextable

np.set_printoptions(suppress=True)

h = 2/256

In [27]:

standard_coord = pd.read_csv("/Users/aymericvie/Documents/GitHub/evology/evology/research/TransferStatus/GainMatrixSingle/data/neutral_static/static_standard.csv")
NT_bump_coord = pd.read_csv("/Users/aymericvie/Documents/GitHub/evology/evology/research/TransferStatus/GainMatrixSingle/data/neutral_static/static_NT_bump.csv")
VI_bump_coord = pd.read_csv("/Users/aymericvie/Documents/GitHub/evology/evology/research/TransferStatus/GainMatrixSingle/data/neutral_static/static_VI_bump.csv")
TF_bump_coord = pd.read_csv("/Users/aymericvie/Documents/GitHub/evology/evology/research/TransferStatus/GainMatrixSingle/data/neutral_static/static_TF_bump.csv")

In [28]:
def write_signif(res):
    if res['p-val'][0] < 0.01:
        # signif = str.maketrans('***')
        signif = '^{***}'
    elif res['p-val'][0] < 0.05:
        # signif = str.maketrans('**')
        signif = '^{**}'
    elif res['p-val'][0] < 0.1:
        # signif = str.maketrans('*')
        signif = '^{*}'
    else:
        signif = ''
    return signif

def Test(series, series_std):
    res = pg.ttest(1/h * (series - series_std), 0, correction=False, confidence=0.95)
    signif = write_signif(res)
    ci = res['CI95%'][0]
    return signif, ci

def ComputeSharpe(series):
    return np.nanmean(series) / np.nanstd(series)

def FormatTable(GainMatrix, SignifMatrix, CiMatrix):
    rowsGM = [['', 'NT', 'VI', 'TF'],
    ['NT', str(GainMatrix[0,0]) + str(SignifMatrix[0]), str(GainMatrix[0,1]) + str(SignifMatrix[1]), str(GainMatrix[0,2]) + str(SignifMatrix[2])],
    ['VI', str(GainMatrix[1,0]) + str(SignifMatrix[3]), str(GainMatrix[1,1]) + str(SignifMatrix[4]), str(GainMatrix[1,2]) + str(SignifMatrix[5])],
    ['TF', str(GainMatrix[2,0]) + str(SignifMatrix[6]), str(GainMatrix[2,1]) + str(SignifMatrix[7]), str(GainMatrix[2,2]) + str(SignifMatrix[8])]]

    tableGM = Texttable()
    tableGM.set_cols_align(["C"] * 4)
    tableGM.set_deco(Texttable.HEADER | Texttable.VLINES | Texttable.BORDER)
    tableGM.add_rows(rowsGM)

    rowsCI = [['', 'NT', 'VI', 'TF'],
    ['NT', str(CiMatrix[0]), str(CiMatrix[1]), str(CiMatrix[2])],
    ['VI', str(CiMatrix[3]), str(CiMatrix[4]), str(CiMatrix[5])],
    ['TF', str(CiMatrix[6]), str(CiMatrix[7]), str(CiMatrix[8])]]

    tableCI = Texttable()
    tableCI.set_cols_align(["C"] * 4)
    tableCI.set_deco(Texttable.HEADER | Texttable.VLINES | Texttable.BORDER)
    tableCI.add_rows(rowsCI)

    return tableGM, tableCI
    

In [29]:
''' What we need is a big function that takes as input:
    some data
    a time frequency (daily, monthly, annualised)
And outputs
    The gain matrix 
    With significance terms/stars
    The Sharpe ratios
    '''

def EstimateGainMatrix(standard_coord, NT_bump_coord, VI_bump_coord, TF_bump_coord, tau):

    GainMatrix = np.zeros((3,3))
    SignifMatrix = [0] * 9
    CiMatrix = [0] * 9
    df = pd.DataFrame()

    def Adjust(arr, tau):
        adj_arr = [(((1 + i) ** tau) - 1) for i in arr]
        return adj_arr

    df['S_NT'] = Adjust(standard_coord['NT_DayReturns'], tau)
    df['S_VI'] = Adjust(standard_coord['VI_DayReturns'], tau)
    df['S_TF'] = Adjust(standard_coord['TF_DayReturns'], tau)

    df['NT_NT'] = Adjust(NT_bump_coord['NT_DayReturns'], tau)
    df['NT_VI'] = Adjust(NT_bump_coord['VI_DayReturns'], tau)
    df['NT_TF'] = Adjust(NT_bump_coord['TF_DayReturns'], tau)

    df['VI_NT'] = Adjust(VI_bump_coord['NT_DayReturns'], tau)
    df['VI_VI'] = Adjust(VI_bump_coord['VI_DayReturns'], tau)
    df['VI_TF'] = Adjust(VI_bump_coord['TF_DayReturns'], tau)

    df['TF_NT'] = Adjust(TF_bump_coord['NT_DayReturns'], tau)
    df['TF_VI'] = Adjust(TF_bump_coord['VI_DayReturns'], tau)
    df['TF_TF'] = Adjust(TF_bump_coord['TF_DayReturns'], tau)

    standard_coord_NT_Return = np.nanmean(df['S_NT'])
    standard_coord_VI_Return = np.nanmean(df['S_VI'])
    standard_coord_TF_Return = np.nanmean(df['S_TF'])

    NT_bump_NT_Return = np.nanmean(df['NT_NT'])
    NT_bump_VI_Return = np.nanmean(df['NT_VI'])
    NT_bump_TF_Return = np.nanmean(df['NT_TF'])

    VI_bump_NT_Return = np.nanmean(df['VI_NT'])
    VI_bump_VI_Return = np.nanmean(df['VI_VI'])
    VI_bump_TF_Return = np.nanmean(df['VI_TF'])

    TF_bump_NT_Return = np.nanmean(df['TF_NT'])
    TF_bump_VI_Return = np.nanmean(df['TF_VI'])
    TF_bump_TF_Return = np.nanmean(df['TF_TF'])
    

    GainMatrix[0,0] = round(1/h * (NT_bump_NT_Return - standard_coord_NT_Return),3)
    GainMatrix[0,1] = round(1/h * (VI_bump_NT_Return - standard_coord_NT_Return),3)
    GainMatrix[0,2] = round(1/h * (TF_bump_NT_Return - standard_coord_NT_Return),3)

    GainMatrix[1,0] = round(1/h * (NT_bump_VI_Return - standard_coord_VI_Return),3)
    GainMatrix[1,1] = round(1/h * (VI_bump_VI_Return - standard_coord_VI_Return),3)
    GainMatrix[1,2] = round(1/h * (TF_bump_VI_Return - standard_coord_VI_Return),3)

    GainMatrix[2,0] = round(1/h * (NT_bump_TF_Return - standard_coord_TF_Return),3)
    GainMatrix[2,1] = round(1/h * (VI_bump_TF_Return - standard_coord_TF_Return),3)
    GainMatrix[2,2] = round(1/h * (TF_bump_TF_Return - standard_coord_TF_Return),3)

    SignifMatrix[0], CiMatrix[0] =  Test(df['NT_NT'], df['S_NT'])
    SignifMatrix[1], CiMatrix[1] =  Test(df['VI_NT'], df['S_NT'])
    SignifMatrix[2], CiMatrix[2] =  Test(df['TF_NT'], df['S_NT'])

    SignifMatrix[3], CiMatrix[3] =  Test(df['NT_VI'], df['S_VI'])
    SignifMatrix[4], CiMatrix[4] =  Test(df['VI_VI'], df['S_VI'])
    SignifMatrix[5], CiMatrix[5] =  Test(df['TF_VI'], df['S_VI'])

    SignifMatrix[6], CiMatrix[6] =  Test(df['NT_TF'], df['S_TF'])
    SignifMatrix[7], CiMatrix[7] =  Test(df['VI_TF'], df['S_TF'])
    SignifMatrix[8], CiMatrix[8] =  Test(df['TF_TF'], df['S_TF'])

    return GainMatrix, SignifMatrix, CiMatrix

In [30]:
print('Gain matrix for daily returns')

GainMatrix, SignifMatrix, CiMatrix = EstimateGainMatrix(standard_coord, NT_bump_coord, VI_bump_coord, TF_bump_coord, 1)
tableGM, tableCI = FormatTable(GainMatrix, SignifMatrix, CiMatrix)

print('\nTexttable Table:')
print(tableGM.draw())
print(latextable.draw_latex(tableGM, 
        caption="Gain matrix at the equal wealth coordinates. Significance is showed for p-value inferior to 0.01 (***), 0.05 (**) and 0.1 (*)."))

print('\nTexttable Table:')
print(tableCI.draw())
print(latextable.draw_latex(tableCI, 
        caption="Confidence intervals at 95\% for the gain matrix elements at the equal wealth coordinates."))

Gain matrix for daily returns

Texttable Table:
+----+-------------+--------------+-------------+
|    |     NT      |      VI      |     TF      |
+====+=============+==============+=============+
| NT | 0.001       | 0            | 0.002^{*}   |
| VI | 0.278^{***} | -0.386^{***} | 0.029^{***} |
| TF | 0.032^{**}  | 0.034^{**}   | -0.021      |
+----+-------------+--------------+-------------+
\begin{table}
	\begin{center}
		\begin{tabular}{|C|C|C|C|}
			\hline
			 & NT & VI & TF \\
			\hline
			NT & 0.001 & 0 & 0.002^{*} \\
			VI & 0.278^{***} & -0.386^{***} & 0.029^{***} \\
			TF & 0.032^{**} & 0.034^{**} & -0.021 \\
			\hline
		\end{tabular}
	\end{center}
	\caption{Gain matrix at the equal wealth coordinates. Significance is showed for p-value inferior to 0.01 (***), 0.05 (**) and 0.1 (*).}
\end{table}

Texttable Table:
+----+-------------+---------------+---------------+
|    |     NT      |      VI       |      TF       |
+====+=============+===============+===============+
| NT 

In [31]:
print('Gain matrix for daily returns')

GainMatrix, SignifMatrix, CiMatrix = EstimateGainMatrix(standard_coord, NT_bump_coord, VI_bump_coord, TF_bump_coord, 21)
tableGM, tableCI = FormatTable(GainMatrix, SignifMatrix, CiMatrix)

print('\nTexttable Table:')
print(tableGM.draw())
print(latextable.draw_latex(tableGM, 
        caption="Gain matrix at the equal wealth coordinates. Significance is showed for p-value inferior to 0.01 (***), 0.05 (**) and 0.1 (*)."))

print('\nTexttable Table:')
print(tableCI.draw())
print(latextable.draw_latex(tableCI, 
        caption="Confidence intervals at 95\% for the gain matrix elements at the equal wealth coordinates."))

Gain matrix for daily returns

Texttable Table:
+----+-------------+--------------+-------------+
|    |     NT      |      VI      |     TF      |
+====+=============+==============+=============+
| NT | 0.061^{***} | -0.008       | 0.137^{***} |
| VI | 2.073^{***} | -1.654^{***} | 0.694^{***} |
| TF | 0.165^{***} | 0.102^{*}    | 0.123^{**}  |
+----+-------------+--------------+-------------+
\begin{table}
	\begin{center}
		\begin{tabular}{|C|C|C|C|}
			\hline
			 & NT & VI & TF \\
			\hline
			NT & 0.061^{***} & -0.008 & 0.137^{***} \\
			VI & 2.073^{***} & -1.654^{***} & 0.694^{***} \\
			TF & 0.165^{***} & 0.102^{*} & 0.123^{**} \\
			\hline
		\end{tabular}
	\end{center}
	\caption{Gain matrix at the equal wealth coordinates. Significance is showed for p-value inferior to 0.01 (***), 0.05 (**) and 0.1 (*).}
\end{table}

Texttable Table:
+----+-------------+---------------+-------------+
|    |     NT      |      VI       |     TF      |
+====+=============+===============+=========

In [32]:
print('Gain matrix for daily returns')

GainMatrix, SignifMatrix, CiMatrix = EstimateGainMatrix(standard_coord, NT_bump_coord, VI_bump_coord, TF_bump_coord, 252)
tableGM, tableCI = FormatTable(GainMatrix, SignifMatrix, CiMatrix)

print('\nTexttable Table:')
print(tableGM.draw())
print(latextable.draw_latex(tableGM, 
        caption="Gain matrix at the equal wealth coordinates. Significance is showed for p-value inferior to 0.01 (***), 0.05 (**) and 0.1 (*)."))

print('\nTexttable Table:')
print(tableCI.draw())
print(latextable.draw_latex(tableCI, 
        caption="Confidence intervals at 95\% for the gain matrix elements at the equal wealth coordinates."))

Gain matrix for daily returns

Texttable Table:
+----+--------------+--------------+--------------+
|    |      NT      |      VI      |      TF      |
+====+==============+==============+==============+
| NT | 12.783^{***} | -2.696^{***} | 31.625^{***} |
| VI | 16.129^{***} | -4.519^{***} | 33.155^{***} |
| TF | 12.445^{***} | -2.611^{***} | 30.814^{***} |
+----+--------------+--------------+--------------+
\begin{table}
	\begin{center}
		\begin{tabular}{|C|C|C|C|}
			\hline
			 & NT & VI & TF \\
			\hline
			NT & 12.783^{***} & -2.696^{***} & 31.625^{***} \\
			VI & 16.129^{***} & -4.519^{***} & 33.155^{***} \\
			TF & 12.445^{***} & -2.611^{***} & 30.814^{***} \\
			\hline
		\end{tabular}
	\end{center}
	\caption{Gain matrix at the equal wealth coordinates. Significance is showed for p-value inferior to 0.01 (***), 0.05 (**) and 0.1 (*).}
\end{table}

Texttable Table:
+----+---------------+---------------+---------------+
|    |      NT       |      VI       |      TF       |
+====+=